In [1]:
import sys
!{sys.executable} -m pip install psycopg2
import psycopg2

userName = None
passWord = None

        

            
def insert_X(Y):

    sql = """INSERT INTO X(Y)
             VALUES(%s) RETURNING Z;"""
    conn = None
    Z = None
    try:
        conn = psycopg2.connect(host="localhost",database="testdb", user="postgres", password="postgres")
        cur = conn.cursor()
        
        
        cur.execute(sql, (Y,))
        
        Z = cur.fetchone()
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    return Z
            
    
def log_in():
    conn = None
    userName = input("Username: ")
    passWord = input("Password: ")
    
    try:
        # connect to PostgreSQL
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
             
    #Handle exceptions    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    #Ensure closed
    finally:
        if conn is not None:
            conn.close()
            
def create_roles():

def create_user (user_name):
    
def grant_privs (role, user_name):
    
def setup_tables ():
    commands = ( """
        CREATE TYPE stENUM AS ENUM ('Hourly', 'Salary');
        ""","""
        CREATE TYPE pENUM AS ENUM ('Hourly', 'Salary');
        ""","""
        create table customer (
        customerId int,
        firstName varchar(20),
        lastName varchar(20),
        Primary key (customerID)
        );
        ""","""
        create table composes(
        ID int,
        mNumber	int,
        foreign key (ID) references Inventory(ID),
        foreign key (mNumber) references Model(mNumber),
        Primary key (ID, mNumber)
        );
        ""","""
        create table inventory(
        ID int,
        cost numeric(8,2),
        leadTime interval,
        category varchar(32),
        Primary key (ID)
        );
        ""","""
        Create table Employee (
        firstName varchar(32),
        lastName varchar(32),
        ssn int,
        employeeID int,
        Salary int,
        salaryType stENUM,
        userID int,
        priviledge pENUM,
        loginTime timestamp,
        logoutTime timestamp,
        Primary key (employeeID)
        );
        ""","""
        Create table model(
        mNumber int,
        salePrice numeric(8,2),
        Primary key (mNumber)
        );
        ""","""
        create table orders (
        orderNumber int,
        quantitySold int,
        saleValue int,
        Primary key (orderNumber),
        employeeID int,
        customerID int,
        mNumber int,
        Foreign key (employeeID) references Employee(employeeID),
        Foreign key (customerID) references Customer(customerID),
        Foreign key (mNumber) references Model(mNumber)
        );
        """
    )
    
    conn = None    
    try:
        # connect to PostgreSQL
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        cur = conn.cursor()
        
        # create each table
        for command in commands:
            cur.execute(command)
            
        cur.close()
        conn.commit()
        
    #Handle exceptions    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    #Ensure closed
    finally:
        if conn is not None:
            conn.close()

def build_rev_report(orderID):
    conn = None
    try:
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        cur = conn.cursor()
        
        
        cur.execute("SELECT saleValue, employeeID, customerID from Orders")
        
        row = cur.fetchone()
        saleValue = row[0]
        employeeID = row[1]
        customerID = row[2]
        print("Employee %s was responsible for $ %s in revenue from customer %s" % (employeeID, saleValue, customerID))
        
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
def build_cust_report(cID): 
    conn = None
    try:
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        cur = conn.cursor()
        
        sql = "SELECT mNumber, SUM(quantitySold), customerID FROM order WHERE customerID = %s GROUP BY mNumber"
        cur.execute(sql, (cID,))
        
        row = cur.fetchone()
        mNumber = row[0]
        quantitySold = row[1]
        customerID = row[2]
        print("Customer %s purchased %s units of model %s" % (customerID, numSold, mNumber ))
        
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
def build_inv_report():
def build_expense_report():
def view_rev_report():
def view_cust_report():
def view_inv_report():
def view_expense_report():

def view_customer(customer_id):
def update_customer(customer_id):
def create_order():

def access_model(model_id):
def update_model(model_id,):
def access_inventory():
def update_inventory():
def view_employees_limited():

def access_employee(employee_id):
def update_employee(employee_id):
def view_employee_sales(employee_id):


    
    
if __name__ == '__main__':
    create_tables()


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Username: postgres
Password: postgres
FATAL:  database "testdb" does not exist

